In [ ]:
#| default_exp yolo

In [ ]:
import textwrap
import json
import openai
import torch
from llmcam.fn_to_fc import complete, tool_schema, form_msg, capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file

In [ ]:
#| export

def detect_objects(
        image_path: str, # Path/URL of image
        conf: float=0.05 # Confidence threshold
    ) -> str: # JSON format of detection results
    """Run YOLO object detection on an input image."""
    model = torch.hub.load("ultralytics/yolov5", "yolov5s")
    model.conf = conf  # NMS confidence threshold
    
    results = model(image_path)
    count = results.pandas().xyxy[0]['name'].value_counts().to_dict()
    print(count)
    return json.dumps(count, indent=2)

In [ ]:
YTLiveTools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file, detect_objects)]

In [ ]:
def exec_func(name, tools=[], **kwargs):
    res = globals().get(name)(**kwargs)
    return res

In [ ]:
#| eval: false

messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    form_msg("user", "Hi, can you capture YouTube Live?")
]
messages.append(form_msg("user", "You can use the default link."))
messages.append(form_msg("user", "Can you extract information from this file?"))
messages.append(form_msg("user", "Use your new tool to detect."))
complete(messages, YTLiveTools, exec_func)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)